# The purpose of this notebook is to test the functions in the various files. It's like a unittest code but I don't know how to do that so I'm doing this instead. 

In [1]:
import sys
import importlib
sys.path.append('.')

%load_ext autoreload
%autoreload 1

## Agents.py testing 

In [2]:
%aimport agents

In [3]:
importlib.reload(agents)

<module 'agents' from '/Users/lucas.spangher/Documents/Berkeley Files/Research/Deferable Loads/transactive_control/agents.py'>

In [4]:
bobs_energy = [500,560,570,580,480,640]
bobs_points = [1,1,1,3,3,3]

Bob = agents.Person(bobs_energy)
Bob_energy_output = Bob.energy_output_simple_linear(bobs_points)
Bob_energy_output

TypeError: list indices must be integers or slices, not str

## Rewards testing

In [5]:
%aimport reward
import osqp
import cvxopt
import numpy as np
import pandas as pd

In [6]:
prices = [1,1,.5,.5,.3,.3]

bobs_reward = reward.Reward(Bob_energy_output, prices, 100, 600)

NameError: name 'Bob_energy_output' is not defined

In [ ]:
bobs_ideal_demands= bobs_reward.ideal_use_calculation()

In [7]:
bobs_reward.neg_distance_from_ideal(bobs_ideal_demands)

NameError: name 'bobs_reward' is not defined

## Testing the setting up of the environment

In [8]:
baseline_energy1 = pd.read_csv("wg1.txt", sep = "\t")
baseline_energy1["net_energy_use"] = baseline_energy1["TOTAL_ENERGY_DESK_TODAY"].diff()
baseline_energy1.net_energy_use.loc[baseline_energy1.net_energy_use<0] = 0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
baseline_energy1.head()

,Id,TimeStamp,WG_1_DSK_3,TOTAL_ENERGY_DESK_TODAY,net_energy_use
0,1,7/30/2018 12:00:00 AM,DESK-3,383.3700,NaN
1,2,7/30/2018 12:15:00 AM,DESK-3,387.7375,4.3675
2,3,7/30/2018 12:30:00 AM,DESK-3,392.1050,4.3675
3,4,7/30/2018 12:45:00 AM,DESK-3,396.4725,4.3675
4,5,7/30/2018 1:00:00 AM,DESK-3,400.8400,4.3675


In [10]:
from datetime import datetime

baseline_energy1["TimeStamp"] = pd.to_datetime(baseline_energy1.TimeStamp)
baseline_energy1.TimeStamp[1:5]

1   2018-07-30 00:15:00
2   2018-07-30 00:30:00
3   2018-07-30 00:45:00
4   2018-07-30 01:00:00
Name: TimeStamp, dtype: datetime64[ns]

In [ ]:
temp_df = baseline_energy1
temp_df["month"] = [str(t.month) if t.month>9 else \
    "0" + str(t.month) for t in baseline_energy1["TimeStamp"]]
temp_df["year"] = [str(t.year) for t in baseline_energy1["TimeStamp"]]
temp_df["hour"] = [str(t.hour) if t.hour>9 else \
    "0" + str(t.hour) for t in baseline_energy1["TimeStamp"]]
temp_df["day"] = [str(t.day) if t.day>9 else \
    "0" + str(t.day) for t in baseline_energy1["TimeStamp"]]
baseline_energy1["date_hour"] =  (temp_df[['year', 'month',"day", "hour"]]
                                  .apply(lambda x: ''.join(x), axis=1))

In [ ]:
b_grouped = baseline_energy1.groupby("date_hour")
# for name, group in b_grouped:
#     print(name)
#     print(group)

final = b_grouped.agg({"Id":"first",
              "TimeStamp": "first",
              "WG_1_DSK_3": "first",
              "TOTAL_ENERGY_DESK_TODAY":"first",
              "net_energy_use":"sum"})

In [ ]:
%matplotlib inline

baseline_energy1.hist("net_energy_use", bins =100)

In [ ]:
%aimport utils

In [ ]:
baseline_energy1 = pd.read_csv("wg1.txt", sep = "\t")
baseline_energy2 = pd.read_csv("wg2.txt", sep = "\t")
baseline_energy3 = pd.read_csv("wg3.txt", sep = "\t")

be1 = utils.change_wg_into_diff(baseline_energy1)
be2 = utils.change_wg_into_diff(baseline_energy2)
be3 = utils.change_wg_into_diff(baseline_energy3)

In [ ]:
be2.head()["net_energy_use"]

In [ ]:
be2[1:1000].plot(x="TimeStamp", y="net_energy_use")



## Test the pricing function

In [30]:
import simulation
import controller

import importlib
importlib.reload(simulation)
importlib.reload(controller)


<module 'controller' from '/Users/lucas.spangher/Documents/Berkeley Files/Research/Deferable Loads/transactive_control/controller.py'>

In [31]:
office=simulation.Office()

creating agents


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


creating controller


In [32]:
p = office.price_signal(1)
office.step(p)

The problem is QP:  True
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 24, constraints m = 49
          nnz(P) + nnz(A) = 72
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -5.1602e+00   1.17e+03   3.56e+04   1.00e-01   8.59e-05s
 200   1.1585e+02   7.48e-02   1.01e-04   5.34e-04   3.01e-04s
 400   1.1572e+02   2.9

(-115017912.27301073, True)

In [ ]:
con = controller.BaseController()
points = con.get_points(p)
pd.DataFrame(points).iloc[0]